![workshops](images/codete_workshops.jpg)

# Introduction to bots

Bots are just a new interface for users. Currently, we are facing a bot revolution, where many companies has not just a web and mobile interface, but also a conversational interface to connect with the users. 

![conversational](images/interface.png)

There are plenty of bot types. One of the taxonomy is presented in [Designing bots](http://shop.oreilly.com/product/0636920057741.do) and looks like following:
* personal bots/team bots,
* voice/text bots,
* super bots and domain-specific bots,
* business and consumer bots,
* net new and integration exposing legacy systems bots.

Developing bots need to be done carefully as it can end as Tay bot, a teenager Twitter bot developed by Microsoft.

![Tay](images/tay.jpeg)

Bots can also have an impact on real-life cases like the bots used during the U.S. elections:
[Bots and trolls in U.S. elections](https://www.nytimes.com/video/us/politics/100000005414346/how-russian-bots-and-trolls-invade-our-lives-and-elections.html)

## Setup Slack account for RTM
It's good to create a new slack workspace, but you can use also a slack account where you have the admin priviliges. Follow the instructions to setup your Slack account for RTM API:

Go to www.slack.com and register.
Create a new workspace.
Click Browser in the top.
Browser for Bots and choose it.
Add configuration
Fill out the fields:
Customize Name
Full Name
Copy the API Token and into a variable SLACK_TOKEN.
Save Integration.

In [ ]:
SLACK_TOKEN = ""

In [ ]:
sc = SlackClient(SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="Hello. I'm Anna, your HR assistant. How can I help?", as_user=True)
else:
    print("Connection failed")

## Natural Language Processing in a nutshell

Based on [O'Reilly](https://www.oreilly.com/ideas/how-to-solve-90-of-nlp-problems--a-step-by-step-guide), we have a process that is known to solve most of the NLP problems.

1. Gather your data
2. Clean yout data
3. Find a good representation
4. Classification
5. Inspection
6. Accounting for vocabulary structure
7. Leveraging semantics
8. Leveraging syntax using end-to-end approaches

### Regular expression

The first solution that comes to our mind when it comes to text processing are regular expression. It is usually the best solution in software development. It is also used in many tools that we have mentioned before. Let's take an example to do a short recap how to use regular expressions in Python.

In [ ]:
example = "Codete is a great company with many great developers."

import re

pattern = "\\s+"
words = re.split(pattern, example)
print(words)

Python does have some methods for Strings that can do the same as above, but with less code.

In [ ]:
words = example.split(' ')
print(words)

NLTK is a more advanced tool than just regular expressions. We can easily tokenize sentences. Tokenization is a process of getting words from sentences.

In [ ]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

tokens = nltk.word_tokenize(example)
print("Tokens: " + str(tokens))

What is great with NLTK is that we can tag each word. A tag give us a meaning of a word. We can think about it as a type of word.

In [ ]:
tags = nltk.pos_tag(tokens)
print("Tagged: " + str(tags))

Some tags are shown in the table below.

| tag  | short | description   |
|------|-------|---------------|
| DT   | determiner | all an another any both del each either every half la many much nary neither no some such that the them these this those |
| IN   | preposition or conjunction, subordinating | astride among uppon whether out inside pro despite on by throughout below within for towards near behind atop around if like until below next into if beside ... |
| JJ   | adjective or numeral, ordinal | third ill-mannered pre-war regrettable oiled calamitous first separable ectoplasmic battery-powered participatory fourth still-to-be-named multilingual multi-disciplinary ... |
| NN   | noun, common, singular or mass | common-carrier cabbage knuckle-duster Casino afghan shed thermostat investment slide humour falloff slick wind hyena override subhumanity machinist ... |
| NNP  | noun, proper, singular | Motown Venneboerger Czestochowa Ranzer Conchita Trumplane Christos Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA Shannon A.K.C. Meltex Liverpool ... |
| NNS  | noun, common, plural | undergraduates scotches bric-a-brac products bodyguards facets coasts divestitures storehouses designs clubs fragrances averages subjectivists apprehensions muses factory-jobs ... |
| VBZ  | verb, present tense, 3rd person singular | bases reconstructs marks mixes displeases seals carps weaves snatches slumps stretches authorizes smolders pictures emerges stockpiles seduces fizzes uses bolsters slaps speaks pleads ... |

## Speech bots

Speech bots are more complex than chatbots and there are just a few platforms that enables this kind of functionality. We use Alexa here to show how we can use it.

### AVS setup

We need to setup the variables needed to connect to AVS in the first place. Fill out the variables below. To do so, we need to go to ...
1. Register at [Alexa Voice Service](https://developer.amazon.com/alexa-voice-service).
2. Go to **AVS Console**.
3. Create a **Product**.
4. Fill out the form: **Product name**, **Product ID**, set Please select your product type to **Alexa-Enabled Application**, set also the **Product category** and **Brief product description**. Answer for the questions: How will end users interact with your product? **Hands-free** and **Far-field**. Do you intend to distribute this product commercially? **No**.
5. Create a **security profile**.
6. Fill out **Allowed origins** with http://localhost:9000/ and **Allowed return URLs** with http://localhost:9000/authresponse.
7. Copy the Product ID (Amazon ID), Client ID and Client Secret to variables below.


In [ ]:
PRODUCT_ID = ""
CLIENT_ID = ""
CLIENT_SECRET = ""
WEB_PORT = 9000

Now, we need to authorize and get the refresh token.

In [ ]:
# source: https://github.com/ewenchou/alexa-client
import cherrypy
import json
import requests
import urllib
import uuid

class Start(object):
    def index(self):
        sd = json.dumps({
            "alexa:all": {
                "productID": PRODUCT_ID,
                "productInstanceAttributes": {
                    "deviceSerialNumber": uuid.getnode()
                }
            }
        })
        url = "https://www.amazon.com/ap/oa"
        callback = "http://localhost:9000/authresponse"#cherrypy.url() + "callback"
        payload = {
            "client_id": CLIENT_ID,
            "scope": "alexa:all",
            "scope_data": sd,
            "response_type": "code",
            "redirect_uri": callback
        }
        req = requests.Request('GET', url, params=payload)
        p = req.prepare()
        raise cherrypy.HTTPRedirect(p.url)

    def authresponse(self, var=None, **params):
        code = urllib.parse.quote(cherrypy.request.params['code'])
        callback = cherrypy.url()
        payload = {
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "code": code,
            "grant_type": "authorization_code",
            "redirect_uri": callback
        }
        url = "https://api.amazon.com/auth/o2/token"
        r = requests.post(url, data=payload)
        resp = r.json()
        return "Refresh token:<br>{}".format(
            resp['refresh_token'])

    index.exposed = True
    authresponse.exposed = True

cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': WEB_PORT})
print('Open http://localhost:9000 to login in amazon alexa service')
cherrypy.quickstart(Start())

Now copy-paste the token below.

In [ ]:
REFRESH_TOKEN=""

We are ready to connect to AVS and get the response from Alexa.

In [ ]:
from avs_client import AlexaVoiceServiceClient

alexa_client = AlexaVoiceServiceClient(
    client_id=CLIENT_ID,
    secret=CLIENT_SECRET,
    refresh_token=REFRESH_TOKEN,
)
alexa_client.connect()

with open('./hello.wav', 'rb') as f:
    alexa_response_audio = alexa_client.send_audio_file(f)
with open('./output.wav', 'wb') as f:
    f.write(alexa_response_audio)

Let's play it.

In [ ]:
from IPython.display import Audio
sound_file = 'output.wav'

Audio(url=sound_file, autoplay=False)